# Проект для «Викишоп»

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**
* *`text`* текст коментария
* *`toxic`* целевой признак

In [1]:
import pandas as pd
from catboost.text_processing import Tokenizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier, Pool
from sklearn.feature_extraction.text import TfidfVectorizer


RANDOM_STATE=42


In [2]:
import sys
!{sys.executable} -m pip install spacy

# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en
import spacy

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 2.0 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Подготовка

Запуск:

In [3]:
try:
    df = pd.read_csv("C:\\Users\\Sergo\\Documents\\toxic_comments.csv", index_col=[0])
except:
    df = pd.read_csv('/datasets/toxic_comments.csv', index_col=[0])

In [4]:
df.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [6]:
'всего коментов',len(df), 'негативных', df['toxic'].sum()

('всего коментов', 159292, 'негативных', 16186)

In [7]:
#%
df['toxic'].sum()/len(df) 

0.10161213369158527

### Токенизация
Приведение всех слов к нижнему регистру и разбиение

In [8]:
tokeliz = Tokenizer(
    lowercasing = True, #нижний регистр
    separator_type='BySense', 
    token_types=['Word']#разбиение по словам 
)

token_text = [tokeliz.tokenize(text) for text in df['text']]

In [9]:
df['text'] = token_text
df['text']

0         [explanation, why, the, edits, made, under, my...
1         [d'aww, he, matches, this, background, colour,...
2         [hey, man, i'm, really, not, trying, to, edit,...
3         [more, i, can't, make, any, real, suggestions,...
4         [you, sir, are, my, hero, any, chance, you, re...
                                ...                        
159446    [and, for, the, second, time, of, asking, when...
159447    [you, should, be, ashamed, of, yourself, that,...
159448    [spitzer, umm, theres, no, actual, article, fo...
159449    [and, it, looks, like, it, was, actually, you,...
159450    [and, i, really, don't, think, you, understand...
Name: text, Length: 159292, dtype: object

**Удаление часто повторяемых слов(стоп слова)**

In [10]:
nltk.download('stopwords')

stop_words = set(stopwords.words('english')) #список слов

def filter_stop_words(token):
    return list(filter(lambda x: x not in stop_words, token))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#token_text_drop_stop = [filter_stop_words(token) for token in token_text]

In [12]:
df['text'] = df['text'].apply(filter_stop_words)
df['text']

0         [explanation, edits, made, username, hardcore,...
1         [d'aww, matches, background, colour, i'm, seem...
2         [hey, man, i'm, really, trying, edit, war, guy...
3         [can't, make, real, suggestions, improvement, ...
4               [sir, hero, chance, remember, page, that's]
                                ...                        
159446    [second, time, asking, view, completely, contr...
159447          [ashamed, horrible, thing, put, talk, page]
159448    [spitzer, umm, theres, actual, article, prosti...
159449    [looks, like, actually, put, speedy, first, ve...
159450    [really, think, understand, came, idea, bad, r...
Name: text, Length: 159292, dtype: object

*Результат:* нужный эффект достигнут. В результате из фраз получился токенизированный список слов, откинул самые часто повторяющиеся слова. Если смотреть на примере 1 фразы ее вышло сжать до 26 слов (-19). Конечно есть вопросики к некоторым словам, но для первого прогона пойдет. 

### Лемматизация

In [13]:
nltk.download('wordnet')
nltk.download('omw-1.4')
lemm = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [14]:
df

,text,toxic,text_lemm
0,"[explanation, edits, made, username, hardcore,...",0,"[explanation, edits, made, username, hardcore,..."
1,"[d'aww, matches, background, colour, i'm, seem...",0,"[d'aww, match, background, colour, i'm, seemin..."
2,"[hey, man, i'm, really, trying, edit, war, guy...",0,"[hey, man, i'm, really, trying, edit, war, guy..."
3,"[can't, make, real, suggestions, improvement, ...",0,"[can't, make, real, suggestion, improvement, w..."
4,"[sir, hero, chance, remember, page, that's]",0,"[sir, hero, chance, remember, page, that's]"
...,...,...,...
159446,"[second, time, asking, view, completely, contr...",0,"[second, time, asking, view, completely, contr..."
159447,"[ashamed, horrible, thing, put, talk, page]",0,"[ashamed, horrible, thing, put, talk, page]"
159448,"[spitzer, umm, theres, actual, article, prosti...",0,"[spitzer, umm, there, actual, article, prostit..."
159449,"[looks, like, actually, put, speedy, first, ve...",0,"[look, like, actually, put, speedy, first, ver..."


In [15]:
df['text'] = df['text'].apply(lambda x:' '.join(x))

**Исправление**

In [23]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lema(x):
    lema_text = nlp(x)
    return " ".join([token.lemma_ for token in lema_text])

In [24]:
#отображение длительности лемматизации
from tqdm import tqdm
tqdm.pandas(desc="bar")

In [28]:
df['text_lemm'] = df['text'].progress_apply(lema)

bar: 100%|██████████| 159292/159292 [14:06<00:00, 188.24it/s]


In [29]:
df

,text,toxic,text_lemm
0,explanation edits made username hardcore metal...,0,explanation edit make username hardcore metall...
1,d'aww matches background colour i'm seemingly ...,0,d'aww match background colour I be seemingly s...
2,hey man i'm really trying edit war guy constan...,0,hey man I be really try edit war guy constantl...
3,can't make real suggestions improvement wonder...,0,can not make real suggestion improvement wonde...
4,sir hero chance remember page that's,0,sir hero chance remember page that be
...,...,...,...
159446,second time asking view completely contradicts...,0,second time ask view completely contradict cov...
159447,ashamed horrible thing put talk page,0,ashamed horrible thing put talk page
159448,spitzer umm theres actual article prostitution...,0,spitzer umm there s actual article prostitutio...
159449,looks like actually put speedy first version d...,0,look like actually put speedy first version de...


Убрал окончания, множественные числа, можно сказать привели слова к исходной очищенной форме. Можно переходить к обучению.

Вывод: данные обработаны и подготовлены к разделению на выборки и к дальнейшему обучению.

## Обучение
Разделение на тренировочную, валидационную и тестовую выборки

In [30]:
target = df['toxic'].values
features = df['text_lemm']

In [31]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = .2, random_state = 42)
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, shuffle=False, test_size=0.5, random_state = 42)

x_train.shape[0], x_valid.shape[0], x_test.shape[0]

(127433, 15930, 15929)

### Превращение слов в вектор

In [32]:
count_tf_idf = TfidfVectorizer()
tfidf_train = count_tf_idf.fit_transform(x_train)
tfidf_valid = count_tf_idf.transform(x_valid)
tfidf_test = count_tf_idf.transform(x_test)

In [33]:
#from sklearn.feature_extraction.text import TfidfVectorizer

#преобразования списка слов с строку
#text_str = [' '.join(words) for words in text_token_drop_stop_nltk]
#count_tf_idf = TfidfVectorizer(min_df=4)# удаление терминов, встречающиеся в менее чем 4 документах
#tf_idf = count_tf_idf.fit_transform(text_str)

#dictionary = count_tf_idf.get_feature_names_out()
#tf_idf

### Обучение логистической регрессии

In [34]:
model_lr = LogisticRegression() 

parameters = {
    'C': list(range(1,15,3)), 
    'class_weight': [None, 'balanced'], 
    'solver': ['liblinear'],
    'random_state':[RANDOM_STATE]
    }

grid = (GridSearchCV(estimator = model_lr, scoring='f1', param_grid=parameters, cv=3, 
                     n_jobs=-1))

grid_lr = grid.fit(tfidf_train, y_train)
print('лучшие параметры', grid_lr.best_params_)
print('лучший показатель модели', grid_lr.best_score_)

лучшие параметры {'C': 13, 'class_weight': None, 'random_state': 42, 'solver': 'liblinear'}
лучший показатель модели 0.7674753932141697


In [35]:
pred_valid_lr = grid_lr.predict(tfidf_valid)
'f1_score, валидационной выборки для подобранной модели:', f1_score(pred_valid_lr, y_valid)

('f1_score, валидационной выборки для подобранной модели:', 0.775654635527247)

**Вывод:** метрика модели преодалела порог так что ее модно рассматривать в для тестовой выборки

### Обучение модели CatBoost
подберем `learning_rate` для Кэт буста. чтобы не было переобучения.

In [ ]:
model_cat = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.3,
    eval_metric='F1',
    random_state=RANDOM_STATE
)#verbose=100,

#передаем массив данных на которых будет происходить обучение 
train_pool = Pool(data=tfidf_train, label=y_train)
valid_pool = Pool(data=tfidf_valid, label=y_valid)
test_pool = Pool(data=tfidf_test, label=y_test)

model_cat.fit(train_pool, eval_set=valid_pool, verbose=False, plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

**Вывод:** на валидационной выборке показал такой-то результат, но длительность обучения оказалась значительно дольше.

**подбор глубины дерева (`depth`) при помощи `GridSearchCV`**

In [ ]:
#model_cat = CatBoostClassifier(
#    #eval_metric='F1',
 #   random_state=RANDOM_STATE
#    )

#param_grid = {
#    'learning_rate': [0.3],
#    'depth': [4, 6, 8],
#    'iterations': [1000]
#}
#model_cat.grid_search(search_by_train_test_split=True)
#grid_cat = GridSearchCV(estimator=model_cat, param_grid=param_grid,
#                        verbose=3, scoring='f1')
#grid_cat.fit(tfidf_train, y_train)
#print('отобранные параметры', grid_cat.best_params_)

пытался подобрать глубину обучения, но это занимает слишком много времени и не так сильно прибавило к качеству модели, так что я убрал эту часть кода.

**Вывод:** подведение результата, модель Логической регрессии метрика оценки качества F1 показала результат 0.76. Этот результат вполне устраивает и подходит поставленному ТЗ

## Обучение лушчей модели на тестовой выборке
Лучшая модель логической регрессии

In [ ]:
pred_test_lr = grid_lr.predict(tfidf_test)
'f1_score, финальной тестовой выборки для подобранной модели =', f1_score(pred_test_lr, y_test)

## Выводы

При работе с текстом данные необходимо их предварительно обработать при работе с численными значениями такого нет. В этом заключается вся специфика работы с такими данными, что слова нужно «закодировать» и перевести в численный формат чтобы компьютер мог их обрабатывать и работать с ними. 
1.	Разбиение фраз на токены и приведение их к должному виду.
2.	Удаление частых слов
3.	Лемматизация – приведение к начальной форме(удаление множественного числа, окончаний)
4.	Перевод слов в вектор(кодирование)

Выбранная модель имеет метрику качества 0.77.



## Чек-лист проверки
- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны